In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
import torch
import numpy as np
from sbi.inference import prepare_for_sbi, SNLE
from sbi.simulators.linear_gaussian import diagonal_linear_gaussian
import sbi.utils as sbi_utils

### Load theta and x

In [6]:
data = np.load('../results/cleaned_up_data.npz', allow_pickle=True)

x = torch.as_tensor(data['data'], dtype=torch.float32)
theta = torch.as_tensor(data['params'], dtype=torch.float32)

### Load prior min and max and gt

In [7]:
import sys
sys.path.append('../')

In [8]:
from parameter_setup import load_ground_truth_params, load_prior_min, load_prior_max

### Load data to standardize - needed to standardize x_o

In [9]:
#standardize_vals = np.load('../results/standardize_vals.npz')
#x_mean = standardize_vals['data_mean']
#x_std = standardize_vals['data_std']
#
#theta_mean = standardize_vals['theta_mean']
#theta_std = standardize_vals['theta_std']

x_mean = x.mean(dim=0)
x_std = x.std(dim=0)

theta_mean = theta.mean(dim=0)
theta_std = theta.std(dim=0)

In [10]:
x = (x-x_mean) / x_std
theta = (theta-theta_mean) / theta_std

### Load x_o (which I got from running the simulator with Arco's ground truth params)

In [11]:
import numpy as np
x_o = torch.as_tensor(np.load('../results/observation/x_o.npz')['x_o'], dtype=torch.float32)

x_o = (x_o - x_mean) / x_std
x_o = x_o.unsqueeze(0)

### Create dummy simulator and dummy prior

In [12]:
def dummy_simulator(theta):
    return torch.ones(1,35)

dummy_prior = sbi_utils.BoxUniform(torch.as_tensor(load_prior_min()), torch.as_tensor(load_prior_max()))
_bound = torch.sqrt(torch.as_tensor(3.))
dummy_prior_norm = sbi_utils.BoxUniform(-_bound*torch.ones(35), _bound*torch.ones(35))

In [14]:
simulator, prior, x_shape = prepare_for_sbi(dummy_simulator, dummy_prior_norm)

inference = SNLE(
    simulator, 
    prior,
    x_shape,
    external_data=(theta[:1000], x[:1000]), # todo: remove the [:1000] to train on all datasamples
    mcmc_method='slice', 
    density_estimator='nsf'
)

In [16]:
posterior = inference(
    num_rounds=1,
    num_simulations_per_round=0,
    batch_size=100, # default is 50
    stop_after_epochs=3, # default is 20
)

Neural network successfully converged after 26 epochs.


In [ ]:
#import pickle
#with open('../results/posteriors/200616_dummyPosteriorSNL.pickle', 'wb') as handle:
#    pickle.dump(posterior, handle)

### Analyse posterior

In [17]:
posterior = posterior.set_default_x(x=x_o)

In [18]:
posterior._mcmc_method = 'slice'

In [19]:
samples = posterior.sample((1000,), num_chains=20, thin=1, show_progress_bars=True)

# denormalize the samples
samples = samples * theta_std + theta_mean

# convert to list for pickling
samples_list = samples.numpy().tolist()

Process 11:
Process 18:
Process 2:
Process 6:
Process 16:
Process 1:
Process 12:
Process 7:
Process 4:
Process 0:
Process 5:
Process 15:
Process 14:
Process 13:
Process 8:
Process 10:
Process 3:
Process 17:
Process 9:
Process 19:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/michael/anaconda3/envs/sbi/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/michael/anaconda3/envs/sbi/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/michael/anaconda3/envs/sbi/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/michael/anaconda3/envs/sbi/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run(

  File "/home/michael/Documents/sbi/sbi/mcmc/slice.py", line 147, in _log_prob_d
    return -self.potential_fn(
  File "/home/michael/Documents/sbi/sbi/mcmc/slice.py", line 130, in sample
    ) = self._sample_from_conditional(params, dim.item())


KeyboardInterrupt: 

  File "/home/michael/Documents/sbi/sbi/mcmc/slice.py", line 172, in _sample_from_conditional
    _log_prob_d(lower) >= log_height
  File "/home/michael/Documents/sbi/sbi/mcmc/slice.py", line 172, in _sample_from_conditional
    _log_prob_d(lower) >= log_height
  File "/home/michael/Documents/sbi/sbi/mcmc/slice.py", line 162, in _sample_from_conditional
    log_height = _log_prob_d(params[self._site_name].view(-1)[dim]) + torch.log(
  File "/home/michael/Documents/sbi/sbi/mcmc/slice.py", line 147, in _log_prob_d
    return -self.potential_fn(
  File "/home/michael/Documents/sbi/sbi/mcmc/slice.py", line 147, in _log_prob_d
    return -self.potential_fn(
  File "/home/michael/Documents/sbi/sbi/inference/snle/snle_base.py", line 394, in pyro_potential
    log_likelihood = self.likelihood_nn.log_prob(
  File "/home/michael/Documents/sbi/sbi/mcmc/slice.py", line 130, in sample
    ) = self._sample_from_conditional(params, dim.item())
  File "/home/michael/Documents/sbi/sbi/inference/snle/sn

In [36]:
#with open('../results/samples/200616_dummyPosteriorSNL_samples.pickle', 'wb') as handle:
#    pickle.dump(samples_list, handle, protocol=2)

### Posterior predictives
Has to be done in a different virtual env with python 2.